In [183]:
import enum

class Player(enum.Enum):
    black = 1
    white = 2

    @property
    def other(self):
        return Player.black if self == Player.white else Player.white


In [184]:
p1 = Player.black
p2 = Player.white

assert p1.other == Player.white
print(p1.value)

1


In [185]:
from collections import namedtuple

class Point(namedtuple("Point", "row col")):
    def neighbors(self):
        return [
            Point(self.row - 1, self.col),  # Up
            Point(self.row + 1, self.col),  # Down
            Point(self.row, self.col - 1),  # Left
            Point(self.row, self.col + 1),  # Right
        ]

    def vert_neighbors(self):
        return [
            Point(self.row - 1, self.col),  # Up
            Point(self.row + 1, self.col),  # Down
        ]

    def hor_neighbors(self):
        return [
            Point(self.row, self.col - 1),  # Left
            Point(self.row, self.col + 1),  # Right
        ]

    def __deepcopy__(self, memodict={}):
        return self


In [186]:
point = Point(1, 2)

point.neighbors()[0].col

2

In [199]:
PAWN_TO_CHAR = {
    None: '.',
    Player.black: 'B',
    Player.white: 'W',
}

def print_move(board, point):
    char = PAWN_TO_CHAR.get(board[point.row, point.col], '.')
    print(f"Move to {point} with pawn {char}")

def print_board(board):
    for row in board:
        print(' '.join(PAWN_TO_CHAR.get(cell, '.') for cell in row))
    print()

def point_from_coords(row, col):
    return Point(row, col)

def coords_from_point(point):
    return point.row, point.col

def decode(move):
    from_row = move // (11*4*10)
    rem = move % (11*4*10)

    from_col = rem // (4*10)
    rem = rem % (4*10)

    direction = rem // 10
    distance = (rem % 10) + 1

    return from_row, from_col, direction, distance

def encode(from_row, from_col, direction, distance):
    if not (0 <= from_row < 11 and 0 <= from_col < 11):
        raise ValueError("Row and column must be between 0 and 10 inclusive")
    if direction not in [0, 1, 2, 3]:
        raise ValueError("Direction must be one of: 0 (up), 1 (down), 2 (left), 3 (right)")
    if not (0 <= distance <= 10):
        raise ValueError("Distance must be between 0 and 10 inclusive")

    return from_row * (11 * 4 * 10) + from_col * (4 * 10) + direction * 10 + (distance - 1)


import numpy as np

EMPTY = 0
WHITE_PAWN = 2
BLACK_PAWN = 1
KING = 3

class Board:

    def __init__(self, board_size=11):
        assert board_size == 11, "Currently only 11x11 board is supported"
        self.size = board_size

    def reset(self):
        self.set_up_board()
        self.print_board()

    def set_up_board(self):
        self.board = np.zeros((self.size, self.size), dtype=int)
        self.current_player = Player.black
        self.place_king()
        self.initialize_black()
        self.initialize_white()

    def place_king(self):
        center = (self.size // 2)
        center_point = Point(center, center)
        self.board[center_point.row, center_point.col] = KING

    def initialize_black(self):
        # TODO: make the black pawn logic work with different sized boards
        center = (self.size // 2)
        for i in range(3, self.size - 3):
            self.board[0, i] = BLACK_PAWN
            self.board[self.size - 1, i] = BLACK_PAWN
            self.board[i, 0] = BLACK_PAWN
            self.board[i, self.size - 1] = BLACK_PAWN
        self.board[1, center] = BLACK_PAWN
        self.board[self.size - 2, center] = BLACK_PAWN
        self.board[center, 1] = BLACK_PAWN
        self.board[center, self.size - 2] = BLACK_PAWN

    def initialize_white(self):
        center = (self.size // 2)
        center_point = Point(center, center)

        pawns_to_place = (self.size + 1)
        points = [center_point]

        while pawns_to_place > 0:
            neighbours = []
            for x in points:
                # Get all neighbors of the current point
                neighbours.extend(x.neighbors())

            for n in neighbours:
                if self.is_on_board(n) and self.board[n.row, n.col] == 0:
                    self.board[n.row, n.col] = WHITE_PAWN  # Place white pawn
                    pawns_to_place -= 1
                    points.append(n)
                    if pawns_to_place == 0:
                        break

    def is_on_board(self, point):
        return 0 <= point.row < self.size and 0 <= point.col < self.size

    def get_pawn_at(self, point):
        if not self.is_on_board(point):
            return None
        return self.board[point.row, point.col]

    def move_pawn(self, player, move):
        # will need to check: pawn of current player, valid move, capture logic
        # move is legal if it doesn't go off the board and doesn't hop over another pawn
        from_row, from_col, direction, distance = decode(move)
        pawn = self.get_pawn_at(Point(from_row, from_col))
        if pawn != (WHITE_PAWN if player == Player.white else BLACK_PAWN):
            raise ValueError("Invalid pawn for the current player")
        if direction == 0:  # Up
            new_row = from_row - distance
            new_col = from_col
        elif direction == 1:  # Down
            new_row = from_row + distance
            new_col = from_col
        elif direction == 2:  # Left
            new_row = from_row
            new_col = from_col - distance
        elif direction == 3:  # Right
            new_row = from_row
            new_col = from_col + distance
        else:
            raise ValueError("Invalid direction")

        new_point = Point(new_row, new_col)
        if not self.is_on_board(new_point):
            raise ValueError("Move goes off the board")
        if self.get_pawn_at(new_point) != EMPTY:
            raise ValueError("Cannot move to a point that is already occupied by another pawn")
        self.board[from_row, from_col] = EMPTY
        self.board[new_point.row, new_point.col] = pawn
        self._check_for_capture(new_point, player)

    def will_capture(self, neighbor_point, player, capture_point):
        new_point = neighbor_point
        if self.board[new_point.row, new_point.col] == player.value:
            self.capture(capture_point)
        elif not self.is_on_board(new_point):
            self.capture(capture_point)

    def capture(self, point):
        self.board[point.row, point.col] = EMPTY


    def _check_for_capture(self, new_point, player):
        opponent_pawn = player.other.value
        # Check if the new point is adjacent to an opponent's pawn
        d = 0 # 0:up, 1:down, 2:left, 3:right
        for neighbor in new_point.neighbors():
            if (self.is_on_board(neighbor) and
                    self.board[neighbor.row, neighbor.col] == opponent_pawn):

                if d == 0:  # Up
                    self.will_capture(Point(neighbor.row - 1, neighbor.col), player, neighbor)

                elif d == 1:  # Down
                    self.will_capture(Point(neighbor.row + 1, neighbor.col), player, neighbor)

                elif d == 2:  # Left
                    self.will_capture(Point(neighbor.row, neighbor.col - 1), player, neighbor)

                elif d == 3:  # Right
                    self.will_capture(Point(neighbor.row, neighbor.col + 1), player, neighbor)

            d += 1


    def get_valid_moves(self, player):
        pass

    def is_king(self, point):
        return self.board[point.row, point.col] == KING

    def print_board(self):
        symbols = {0: '.', 1: 'B', 2: 'W', 3: 'K'}
        for row in self.board:
            print(' '.join(symbols[cell] for cell in row))
        print()


board = Board(11)
board.reset()
board.move_pawn(Player.black, encode(3,0, 3, 4))
board.print_board()
board.move_pawn(Player.black, encode(0, 6, 1, 3))
board.print_board()




. . . B B B B B . . .
. . . . . B . . . . .
. . . . . . . . . . .
B . . . . W . . . . B
B . . . W W W . . . B
B B . W W K W W . B B
B . . . W W W . . . B
B . . . . W . . . . B
. . . . . . . . . . .
. . . . . B . . . . .
. . . B B B B B . . .

. . . B B B B B . . .
. . . . . B . . . . .
. . . . . . . . . . .
. . . . B W . . . . B
B . . . W W W . . . B
B B . W W K W W . B B
B . . . W W W . . . B
B . . . . W . . . . B
. . . . . . . . . . .
. . . . . B . . . . .
. . . B B B B B . . .

. . . B B B . B . . .
. . . . . B . . . . .
. . . . . . . . . . .
. . . . B . B . . . B
B . . . W W W . . . B
B B . W W K W W . B B
B . . . W W W . . . B
B . . . . W . . . . B
. . . . . . . . . . .
. . . . . B . . . . .
. . . B B B B B . . .



In [188]:
decode(2323)

(5, 3, 0, 4)

In [156]:
encode(5,3,0,4)

2323

In [144]:
class Move:
    def __init__(self, from_row, from_col, direction, distance):
        pass

class GameState:
    def __init__(self, board, next_player, previous, move):
        self.board = board
        self.next_player = next_player
        self.previous = previous
        self.last_move = move
        self.winner = None

    @classmethod # why classmethod?
    def new_game(cls):
        return cls(Board(), Player.black, None, None)

    def apply_move(self, move):
        new_board = self.board.copy()
        new_board.move_pawn(self.next_player, move)

        next_state = GameState(new_board, self.next_player.other, self, move)

        next_state.is_over()

        return next_state


    @property
    def situation(self):
        return self.board, self.next_player

    def is_legal_move(self, move):
        # Check if the move is legal
        return True

    def is_over(self):
        pass

    def legal_moves(self):
        pass

    def winner(self):
        pass



